<a href="https://colab.research.google.com/github/Muppidathe/ml_project/blob/master/Pipeline_and_ColumnTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [6]:
df=pd.DataFrame({'Gender':['Male','Female','Male','Female','Female'],'city':['chennai','delhi','madurai','coimbatore','nellai']})
df

,Gender,city
0,Male,chennai
1,Female,delhi
2,Male,madurai
3,Female,coimbatore
4,Female,nellai


# with sparse_output=True

In [11]:
one_hot_encode=OneHotEncoder()
res=one_hot_encode.fit_transform(df)
print(res)
print(type(res))

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (5, 7)>
  Coords	Values
  (0, 1)	1.0
  (0, 2)	1.0
  (1, 0)	1.0
  (1, 4)	1.0
  (2, 1)	1.0
  (2, 5)	1.0
  (3, 0)	1.0
  (3, 3)	1.0
  (4, 0)	1.0
  (4, 6)	1.0
<class 'scipy.sparse._csr.csr_matrix'>


# with sparse_output=False

In [13]:
one_hot_encode=OneHotEncoder(sparse_output=False)
res=one_hot_encode.fit_transform(df)
print(res)
print(type(res))

[[0. 1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1.]]
<class 'numpy.ndarray'>


In [15]:
df=pd.DataFrame(res,columns=one_hot_encode.get_feature_names_out())
df

,Gender_Female,Gender_Male,city_chennai,city_coimbatore,city_delhi,city_madurai,city_nellai
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0


# applying `pipeline` and `columnTransformer`

In [27]:
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
df1=pd.read_csv('https://raw.githubusercontent.com/Muppidathe/ml_project/refs/heads/master/dataset_traffic_accident_prediction1.csv')
df1.head()

,Weather,Road_Type,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,Accident
0,Rainy,City Road,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,Artificial Light,0.0
1,Clear,Rural Road,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,0.0
2,Rainy,Highway,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,0.0
3,Clear,City Road,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,0.0
4,Rainy,Highway,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,1.0


In [28]:
df1.dropna(subset=['Accident'],inplace=True)
x=df1.drop(columns=['Accident'])
y=df1['Accident']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
cat_columns=x.select_dtypes(include=['object']).columns
num_columns=x.select_dtypes(include=['int','float']).columns

In [30]:
cat_preprocess_pipe=Pipeline([('mode imputer',SimpleImputer(strategy='most_frequent')),('encoding for cat column',OneHotEncoder())])
num_preprocess_pipe=Pipeline([('mean imputer',SimpleImputer(strategy='mean')),('normalizing values',MinMaxScaler())])
column_transformer=ColumnTransformer(transformers=[('cat transform',cat_preprocess_pipe,cat_columns),('num transform',num_preprocess_pipe,num_columns)])
final_pipeline=Pipeline([('pre processing',column_transformer),('model',RandomForestClassifier(random_state=42))])
final_pipeline.fit(x_train,y_train)

Pipeline(steps=[('pre processing',
                 ColumnTransformer(transformers=[('cat transform',
                                                  Pipeline(steps=[('mode '
                                                                   'imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding '
                                                                   'for cat '
                                                                   'column',
                                                                   OneHotEncoder())]),
                                                  Index(['Weather', 'Road_Type', 'Time_of_Day', 'Accident_Severity',
       'Road_Condition', 'Vehicle_Type', 'Road_Light_Condition'],
      dtype='object')),
                                                 ('num transform',
                                                  Pipeline(steps=[('mean '
                                                                   'imputer',
                                                                   SimpleImputer()),
                                                                  ('normalizing '
                                                                   'values',
                                                                   MinMaxScaler())]),
                                                  Index(['Traffic_Density', 'Speed_Limit', 'Number_of_Vehicles',
       'Driver_Alcohol', 'Driver_Age', 'Driver_Experience'],
      dtype='object'))])),
                ('model', RandomForestClassifier(random_state=42))])